### 0. Install Julia and POMDP.jl

Please first install Julia and POMDP.jl (https://github.com/JuliaPOMDP/POMDPs.jl).

### 1. Include Julia packages and load the problem
Several POMDP julia packages such as "POMDPTools, POMDPModels" are already included in planner.jl.

If there is an error about uninstalled packages, please install them with "pkg.add()".

In [ ]:
include("../ContinuousPOMCGS/Planner.jl")
using Plots
using LaserTag

# --- Laser Tag ---
rng = MersenneTwister(7)
# println(discount(pomdp))
pomdp = gen_lasertag(rng=rng, robot_position_known=false)

### 2. Define parameters

In [ ]:
# --- 0.1: Parameters ---
## This is an exmaple parameter configuraiton ##
b0 = initialstate(pomdp) # Get POMDP's initial belief
nb_process_samples = 10000
max_b_gap = 0.3
nb_particles_b0 = nb_process_samples
c = 1.0
nb_abstract_obs = 5
epsilon = 0.1
particles_b0, dict_weighted_b0 = InitContinuousActionPOMDP(pomdp, b0, nb_particles_b0)
rmin, a_rmin = FindRLower(pomdp, b0, action_space)

### 3. Prepare $V_{mdp}$

Here we use a Q-learning algorithm to compute $V_{MDP}$, one can also use other MDP solvers.

In [ ]:
# define Q learning parameters
Q_table = Dict{Any, Dict{Int64, Float64}}()
learning_rate = 0.9
explore_rate = 0.6


Q_learning_policy = Qlearning(Q_table, learning_rate, explore_rate, action_space)

# --- 0.2: Prepare Rollout Policy ---
nb_episode_size = 30
nb_max_episode = 10
nb_sim = 500
epsilon = 0.01
Training(Q_learning_policy, nb_episode_size, nb_max_episode, nb_sim, epsilon, particles_b0, pomdp)

In [ ]:
# --- 1: Define POMCGS and fsc ---
planner = ContinuousPlannerPOMCGS(rmin, Vmdp)
# planner._bool_PCA_observation = true # if one want to test with PCA dimension reduction
InitPlannerParameters(planner, 
                    nb_process_samples, 
                    max_b_gap,
                    c,
                    discount(pomdp),
                    nb_abstract_obs,
                    false,
                    Vector{Float64}(),
                    epsilon)
fsc = InitFSC(max_b_gap, max_node_size, action_space)

### 4. Init POMCGS and run the solver 

In [ ]:
# --- 2: Graph Search ---
Search(pomdp, particles_b0, dict_weighted_b0, fsc, planner)